## Loading files into mongodb

In [1]:
from pymongo import MongoClient # Install for using this lib.
import subprocess

In [2]:
dbname = 'test'
collection_name1 = 'reviews'
collection_name2 = 'business'
collection_name3 = 'users'

path = "/Users/alvira/Desktop/distributedData/project/"

input_file_name1 = path+"review.json"
input_file_name2 = path+'business.json'
input_file_name3 = path+'user.json'



In [3]:
def import_query(dbname, collection_name, input_file_name):
    mongoimport_query =  'mongoimport --db ' + dbname + ' --collection ' + collection_name + ' --file ' + input_file_name
    #COMPLETE THIS.
    return mongoimport_query
 
#Create connection 
client = MongoClient() #default-localhost:27017
#Connect to database
db = client[dbname]

#Drop table.
db[collection_name1].drop()
db[collection_name2].drop()
db[collection_name3].drop()

#Insert all data from the input_file_name.

In [4]:
mongoimport_query = import_query(dbname, collection_name1, input_file_name1)
subprocess.call(mongoimport_query,shell=True)

mongoimport_query = import_query(dbname, collection_name2, input_file_name2)
subprocess.call(mongoimport_query,shell=True)

mongoimport_query = import_query(dbname, collection_name3, input_file_name3)
subprocess.call(mongoimport_query,shell=True)

0

In [5]:
# db.reviews.find()

## Keep only restaurants

In [5]:
restaurants = db.business.find({ '$or' : [ { "attributes.RestaurantsReservations" : {'$exists' : 'true'} }, { "attributes.RestaurantsDelivery" : {'$exists':'true'}}, 
                            { "attributes.RestaurantsAttire" : {'$exists':'true'}}, { "attributes.RestaurantsCounterService" : {'$exists':'true'}}, 
                            { "attributes.RestaurantsGoodForGroups" : {'$exists':'true'}}, { "attributes.RestaurantsPriceRange2" : {'$exists':'true'}}, 
                            { "attributes.RestaurantsTableService" : {'$exists':'true'}}, { "attributes.RestaurantsTakeOut" : {'$exists':'true'}}, 
                            { "attributes.GoodForMeal" : {'$exists':'true'}}]})

In [ ]:
len(list(restaurants))

#### Finding restuarants having review counts greater than 25

In [5]:
restaurants = db.business.find({ '$or' : [ { "attributes.RestaurantsReservations" : {'$exists' : 'true'} }, { "attributes.RestaurantsDelivery" : {'$exists':'true'}}, 
                            { "attributes.RestaurantsAttire" : {'$exists':'true'}}, { "attributes.RestaurantsCounterService" : {'$exists':'true'}}, 
                            { "attributes.RestaurantsGoodForGroups" : {'$exists':'true'}}, { "attributes.RestaurantsPriceRange2" : {'$exists':'true'}}, 
                            { "attributes.RestaurantsTableService" : {'$exists':'true'}}, { "attributes.RestaurantsTakeOut" : {'$exists':'true'}}, 
                            { "attributes.GoodForMeal" : {'$exists':'true'}}], 
                    "review_count":{"$gt": 100}})

In [6]:
rest = list(restaurants)

In [7]:
# Collecting business ID's
business_ids = []
for i in range(len(rest)):
    business_ids.append(rest[i]['business_id'])

In [8]:
len(business_ids)

8322

In [ ]:
# list(db.business.find({ '$or' : [ { "attributes.RestaurantsReservations" : {'$exists' : 'true'} }, { "attributes.RestaurantsDelivery" : {'$exists':'true'}}, 
#                             { "attributes.RestaurantsAttire" : {'$exists':'true'}}, { "attributes.RestaurantsCounterService" : {'$exists':'true'}}, 
#                             { "attributes.RestaurantsGoodForGroups" : {'$exists':'true'}}, { "attributes.RestaurantsPriceRange2" : {'$exists':'true'}}, 
#                             { "attributes.RestaurantsTableService" : {'$exists':'true'}}, { "attributes.RestaurantsTakeOut" : {'$exists':'true'}}, 
#                             { "attributes.GoodForMeal" : {'$exists':'true'}}]}))

#### Filtering users (having reviews count greater than 25)

In [9]:
filtered_users = db.users.find({"review_count":{"$gt": 50}})

In [10]:
user = list(filtered_users )

In [11]:
user_ids = []
for i in range(len(user)):
    user_ids.append(user[i]['user_id'])

In [12]:
len(user_ids)

112146

#### Filtering reviews basis above user_id & business_id and stars

In [ ]:
# len(list(db.reviews.find()))

In [13]:
filtered_reviews = db.reviews.find({"user_id": {'$in':user_ids}, "business_id": {'$in':business_ids}, 
                                    "stars": {'$in':[1,2,3,4,5]}})

In [45]:
# review = list(filtered_reviews )

In [58]:
# del rest

In [48]:
# len(review)

883429

**Saving filtered reviews as a table:**

In [14]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
from pyspark.sql.functions import *

In [15]:
db.filtered_reviews.drop()
result = db.filtered_reviews.insert_many(filtered_reviews)
# result.inserted_ids

## Load tables into pyspark

In [16]:
reviews = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/test.filtered_reviews").load()

# business = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/test.business").load()

# users = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/test.users").load()

In [52]:
reviews.show(1)

+--------------------+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+
|                 _id|         business_id|cool|      date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+
|[5a5bde777cb40ff2...|mr4FiPaXTWlJ3qGzp...|   0|2009-07-21|    0|Z5l99h18E3_g1GLcD...|    3|I left Table 17 f...|     3|djpMXOA1ic5wv3FPt...|
+--------------------+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+
only showing top 1 row



In [17]:
reviews.count()

883429

In [ ]:
del filtered_reviews, filtered_users, restaurants, user

In [19]:
reviews = reviews.select(['business_id', 'user_id', 'text', 'stars'])

In [25]:
# business = business.select(['business_id', 'stars'])

In [26]:
# users = users.select(['user_id'])

In [27]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
import pyspark.mllib.feature

In [21]:
categorize = udf(lambda (x) : int(x>3))
reviews2 = reviews.withColumn('class',categorize('stars')).select(['text', 'class','stars','business_id', 'user_id'])

In [25]:
# del business_ids, reviews, rest, user_ids

In [53]:
# reviews2.repartition(1).write.json(path+"reviews2.json")

In [126]:
import re, string
regex = re.compile('[^a-zA-Z]')
def words(text):
    c = regex.sub(' ', text)
    lis = re.sub(r'\b\w{1,2}\b', '', c).split()
    return set(map(lambda x: x.lower(), lis))

In [30]:
# tokenize = udf(lambda (x) : words(x))
# reviews2 = reviews2.withColumn("features", tokenize("text"))

In [41]:
# reviews2 = reviews2.select(["class", "text"])

In [26]:
reviews2.groupby("class").count().show()

+-----+------+
|class| count|
+-----+------+
|    0|287851|
|    1|595578|
+-----+------+



In [101]:
reviews2.show()

+--------------------+-----+-----+--------------------+--------------------+
|                text|class|stars|         business_id|             user_id|
+--------------------+-----+-----+--------------------+--------------------+
|I left Table 17 f...|    0|    3|mr4FiPaXTWlJ3qGzp...|djpMXOA1ic5wv3FPt...|
|Ok, full disclosu...|    1|    5|mr4FiPaXTWlJ3qGzp...|XCDxUzBSY8-JOvcp4...|
|for the time bein...|    0|    3|mr4FiPaXTWlJ3qGzp...|-pXs08gJq9ExIk275...|
|Found this place ...|    1|    4|mr4FiPaXTWlJ3qGzp...|Kf10dTiGlcnyhj8DB...|
|Table 17 is a cla...|    0|    3|mr4FiPaXTWlJ3qGzp...|zZy5Jljx7rEvISiJ2...|
|I wasn't super im...|    0|    3|mr4FiPaXTWlJ3qGzp...|Jyi0WJt0UfGdEg0gr...|
|I'm going to igno...|    1|    4|mr4FiPaXTWlJ3qGzp...|JUT0U3HTSB3kz9Wh7...|
|I love this place...|    1|    4|mr4FiPaXTWlJ3qGzp...|AhoxHm569hH_PRkoe...|
|Booked through Op...|    1|    4|mr4FiPaXTWlJ3qGzp...|gXQfe8T1UvMpmR5rc...|
|I came here for w...|    1|    4|mr4FiPaXTWlJ3qGzp...|alUuOskFSl1bODjnc...|

In [27]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer, StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql import Row

In [ ]:
# model = NaiveBayes.train(training, 1.0)
# # Make prediction and test accuracy.
# predictionAndLabel = test.map(lambda p: (model.predict(p.features), p.label))
# accuracy = 1.0 * predictionAndLabel.filter(lambda pl: pl[0] == pl[1]).count() / test.count()
# print('model accuracy {}'.format(accuracy))

In [102]:
training, test = reviews2.select(['text', 'class','stars']).randomSplit([0.8, 0.2])

In [30]:
# test.count()

177207

In [31]:
# training_stars = training.select(["stars"]).collect()

In [32]:
# test_stars = test.select(["stars"]).collect()

In [33]:
# len(test_stars)

176962

In [34]:
# training = training.select(["text","class"])
# test = test.select(["text","class"])

In [39]:
# test.count()

177252

In [134]:
# training.printSchema()

root
 |-- text: string (nullable = true)
 |-- class: string (nullable = true)



In [104]:
categoryIndexer = StringIndexer(inputCol="class", outputCol="label")
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol="words", outputCol="features", numFeatures=20000)
nb = NaiveBayes(smoothing=1.0)

# categoryConverter = IndexToString(inputCol="label", outputCol="predCategory", labels=categoryIndexer.labels)
pipeline = Pipeline(stages=[categoryIndexer, tokenizer, hashingTF, nb])

model = pipeline.fit(training)
pr = model.transform(test)

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
metric = evaluator.evaluate(pr)

print "F1 metric = %g" % metric

F1 metric = 0.813997


In [155]:
from pyspark.sql.types import DoubleType
def prob2star(val): 
    if val < 0.2: return int(1)
    elif val >= 0.2 and val < 0.4: return int(2)
    elif val >= 0.4 and val < 0.6: return int(3)
    elif val >= 0.6 and val < 0.8: return int(4)
    else: return int(5)
#     return val
get_star = udf(lambda (x) : prob2star(x[1]))  

In [156]:
pr2 = pr.withColumn("pred_stars", get_star('probability'))

In [157]:
pr2.select(["stars","pred_stars","probability","label", "prediction"]).take(10)

[Row(stars=4, pred_stars=u'1', probability=DenseVector([0.9991, 0.0009]), label=0.0, prediction=0.0),
 Row(stars=4, pred_stars=u'1', probability=DenseVector([0.9709, 0.0291]), label=0.0, prediction=0.0),
 Row(stars=5, pred_stars=u'1', probability=DenseVector([0.995, 0.005]), label=0.0, prediction=0.0),
 Row(stars=2, pred_stars=u'3', probability=DenseVector([0.4497, 0.5503]), label=1.0, prediction=1.0),
 Row(stars=3, pred_stars=u'1', probability=DenseVector([0.8682, 0.1318]), label=1.0, prediction=0.0),
 Row(stars=5, pred_stars=u'1', probability=DenseVector([0.9949, 0.0051]), label=0.0, prediction=0.0),
 Row(stars=2, pred_stars=u'5', probability=DenseVector([0.0004, 0.9996]), label=1.0, prediction=1.0),
 Row(stars=3, pred_stars=u'1', probability=DenseVector([0.9937, 0.0063]), label=1.0, prediction=0.0),
 Row(stars=4, pred_stars=u'1', probability=DenseVector([0.9104, 0.0896]), label=0.0, prediction=0.0),
 Row(stars=2, pred_stars=u'5', probability=DenseVector([0.1782, 0.8218]), label=1.0,

In [158]:
from pyspark.sql.types import IntegerType
toIntfunc = udf(lambda x: x,IntegerType())
pr2 = pr2.withColumn("pred_stars",toIntfunc("pred_stars"))

In [159]:
pr2.printSchema()

root
 |-- text: string (nullable = true)
 |-- class: string (nullable = true)
 |-- stars: integer (nullable = true)
 |-- label: double (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)
 |-- pred_stars: integer (nullable = true)



In [48]:
import numpy as np
import math
def rmse(x,y):
    return math.sqrt(np.mean((np.array(x)-np.array(y))**2))

In [148]:
#Can soemone help in resolving this error
evaluator = MulticlassClassificationEvaluator(labelCol="stars", predictionCol="pred_stars", metricName="accuracy")
metric = evaluator.evaluate(pr2)
print

IllegalArgumentException: u'requirement failed: Column pred_stars must be of type DoubleType but was actually IntegerType.'

In [162]:
pr2 = pr2.withColumn("square", (pr2["stars"]-pr2["pred_stars"])**2)

In [166]:
mean = pr2.select(mean("square")).collect()

In [172]:
import math
rmse = math.sqrt(mean[0][0])
print rmse

2.98544816538


In [183]:
pr2.select("square").groupby("square").count().show()

+------+-----+
|square|count|
+------+-----+
|   0.0| 8310|
|   1.0|19756|
|   4.0|32826|
|   9.0|61481|
|  16.0|54197|
+------+-----+

